In [1]:
from plotly.offline import download_plotlyjs, init_notebook_mode, plot, iplot
init_notebook_mode(connected=True)

In [2]:
import pandas

train = pandas.read_csv(r'C:\Users\jyellin\re_1\tacred\results\general\train-ud_paths_per_relation_no_relation.csv')
test = pandas.read_csv(r'C:\Users\jyellin\re_1\tacred\results\general\test-ud_paths_per_relation_no_relation.csv')

In [3]:
def transform_row(r):
    return '{entity1} {org_path} {entity2}'.format(entity1=r.type1[0:3], org_path=r.path, entity2=r.type2[0:3])

train['path'] = train.apply(transform_row, axis=1)
test['path'] = test.apply(transform_row, axis=1)

In [4]:
test_total_by_r_and_p = test.groupby(['relation','path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[False,False]).reset_index(drop=True)
train_total_by_r_and_p = train.groupby(['relation','path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['relation','total'], ascending=[True,False]).reset_index(drop=True)
train_total_by_p = train.groupby(['path'])['id'].count().reset_index().rename(columns={'id': 'total'}).sort_values(['total'], ascending=[False]).reset_index(drop=True)


In [5]:
def label_count_in_train_across_relations(test_row):
    path = test_row['path']

    train_row = train_total_by_p.loc[train_total_by_p['path'] == path]
    if train_row.empty:
        return 0
    else:
        compare  = train_row.values[0][1]
        return compare

def label_count_in_train_by_relation(test_row):
    relation = test_row['relation']
    path = test_row['path']

    train_row = train_total_by_r_and_p.loc[
        (train_total_by_r_and_p['relation'] == relation) & 
        (train_total_by_r_and_p['path'] == path)]
    if train_row.empty:
        return 0
    else:
        compare  = train_row.values[0][2]
        return compare

test_total_by_r_and_p['total-for-relation'] = test_total_by_r_and_p.apply(lambda row : label_count_in_train_by_relation(row), axis=1)
test_total_by_r_and_p['total-across-relations'] = test_total_by_r_and_p.apply(lambda row : label_count_in_train_across_relations(row), axis=1)

KeyboardInterrupt: 

In [ ]:
stats = []
relations = train_total_by_r_and_p['relation'].unique().tolist()
for relation in relations:
    
    df = test_total_by_r_and_p[test_total_by_r_and_p['relation']==relation]
    
    matched = sum(df[df['total-for-relation']>0]['total'].to_list())
    matched_lax = sum(df[df['total-across-relations']>0]['total'].to_list())
    total = sum(df['total'].to_list())
    
    stats.append( (relation, round(matched/total,4), round(matched_lax/total,4) ) )

    
matched = sum(test_total_by_r_and_p[test_total_by_r_and_p['total-for-relation']>0]['total'].to_list())
matched_lax = sum(test_total_by_r_and_p[test_total_by_r_and_p['total-across-relations']>0]['total'].to_list())
total = sum(test_total_by_r_and_p['total'].to_list())

stats.insert(0, ('overall', round(matched/total,4) , round(matched_lax/total),2) )
stats = sorted(stats, key=lambda x: x[1], reverse=True)

In [ ]:
import plotly.graph_objs as go

trace1 = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[2] for stat in stats],
    name='Pseudo Recall',
    width=0.5,
    marker=dict(
        color='rgb(158,202,225)',
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

overall_index = next(i for (i, stat) in enumerate(stats) if stat[0] == 'overall')

colors = ['rgb(58,200,225)'] * len(stats)
colors[overall_index] = 'rgb(181,59,89)'


trace2 = go.Bar(
    x=[stat[0] for stat in stats],
    y=[stat[1] for stat in stats],
    name='Recall',
    width=0.5,
    marker=dict(
        color=colors,
        line=dict(
            color='rgb(8,48,107)',
            width=0.5),
        ),
    opacity=0.6
)

data = [trace2]
layout = go.Layout(
    title='UCCA Path - Recall',
    xaxis_title="Relation",
    yaxis_title="Recall",        
    barmode='overlay',
    width=1200,
    xaxis = go.layout.XAxis(
        tickangle = 45,
        automargin = True
        
    )
    
        
)

fig = go.Figure(data=data, layout=layout)
iplot(fig)